# Demo: Train and evaluate a GRU predictor (PyTorch)
This notebook contains a self-contained workflow to train a GRU model on a reservoir time series exported from InfluxDB and evaluate its forecast on a hold-out test period.

It reproduces the logic implemented in `src/gru_predictor.py` but runs inside the notebook for easy experimentation and visualization.

Steps:
1. Import libraries and project utilities
2. Load CSV (InfluxDB-style) using the shared loader
3. Prepare sequences, scale data and create DataLoaders
4. Define GRU model and training loop
5. Train the model and plot training/validation loss
6. Iterative multi-step forecasting on the test horizon
7. Evaluate (MAE, RMSE, MAPE) and plot comparison

In [ ]:
# Cell 2: imports and setup
import sys, os
sys.path.append(os.path.abspath('../src'))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

from plot_single import load_time_series, slice_by_dates

print('Ready. PyTorch available:', torch.__version__)

In [ ]:
# Cell 3: Load data and prepare series
csv_path = os.path.abspath('../data/test.csv')
df = load_time_series(csv_path)
print('Columns:', list(df.columns))

# choose field and optional date range
field = 'reserva'
start = None
end = None
sub = slice_by_dates(df, start, end)
series = pd.to_numeric(sub[field], errors='coerce')
series.index = sub.index
series = series.dropna()
print('Series length:', len(series), 'from', series.index.min(), 'to', series.index.max())

# hyperparameters for demo
test_days = 30
seq_len = 30
hidden_size = 64
num_layers = 1
epochs = 30
batch_size = 32
lr = 1e-3

train = series.iloc[:-test_days]
test = series.iloc[-test_days:]

# scaling
scaler = StandardScaler()
train_vals = train.values.reshape(-1, 1)
scaler.fit(train_vals)
all_scaled = scaler.transform(series.values.reshape(-1, 1)).ravel()
train_scaled = scaler.transform(train.values.reshape(-1,1)).ravel()

# helper to create sequences
def create_sequences(values, seq_len):
    X, y = [], []
    for i in range(len(values) - seq_len):
        X.append(values[i:i+seq_len])
        y.append(values[i+seq_len])
    X = np.array(X)
    y = np.array(y)
    if X.ndim == 2:
        X = X.reshape(X.shape[0], X.shape[1], 1)
    return X, y

X_all, y_all = create_sequences(all_scaled[:len(train_scaled)], seq_len)
# validation split
n_train = int(len(X_all) * 0.9)
X_train, y_train = X_all[:n_train], y_all[:n_train]
X_val, y_val = X_all[n_train:], y_all[n_train:]

print('Train samples:', len(X_train), 'Val samples:', len(X_val))

In [ ]:
# Cell 4: Dataset, model and training functions
class TimeSeriesDataset(Dataset):
    def __init__(self, seqs, targets):
        self.x = seqs.astype(np.float32)
        self.y = targets.astype(np.float32)
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

class GRUModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=1, dropout=0.0):
        super().__init__()
        self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 1)
    def forward(self, x):
        out, _ = self.gru(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out.squeeze(-1)

# training loop returning history
def train_model(model, train_loader, val_loader, device, epochs=20, lr=1e-3):
    model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()
    best_val = np.inf
    best_state = None
    history = {'train_loss': [], 'val_loss': []}
    for ep in range(1, epochs+1):
        model.train()
        train_losses = []
        for xb, yb in train_loader:
            xb = xb.to(device); yb = yb.to(device)
            pred = model(xb)
            loss = loss_fn(pred, yb)
            opt.zero_grad(); loss.backward(); opt.step()
            train_losses.append(loss.item())
        model.eval()
        val_losses = []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(device); yb = yb.to(device)
                pred = model(xb)
                val_losses.append(float(loss_fn(pred, yb)))
        tloss = np.mean(train_losses) if train_losses else np.nan
        vloss = np.mean(val_losses) if val_losses else np.nan
        history['train_loss'].append(tloss); history['val_loss'].append(vloss)
        print(f'Epoch {ep}/{epochs} train_loss={tloss:.6f} val_loss={vloss:.6f}')
        if vloss < best_val:
            best_val = vloss; best_state = model.state_dict()
    if best_state is not None:
        model.load_state_dict(best_state)
    return model, history

# iterative forecasting function
def iterative_forecast(model, seed_seq, steps, device, scaler=None):
    model.to(device); model.eval()
    seq = seed_seq.copy().astype(np.float32)
    preds = []
    with torch.no_grad():
        for _ in range(steps):
            x = torch.tensor(seq.reshape(1, seq.shape[0], 1), dtype=torch.float32).to(device)
            out = model(x).cpu().numpy().ravel()[0]
            preds.append(out)
            seq = np.roll(seq, -1); seq[-1] = out
    preds = np.array(preds)
    if scaler is not None:
        preds = scaler.inverse_transform(preds.reshape(-1,1)).ravel()
    return preds

In [ ]:
# Cell 5: Prepare DataLoaders, build model and train
train_ds = TimeSeriesDataset(X_train, y_train)
val_ds = TimeSeriesDataset(X_val, y_val)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

model = GRUModel(input_size=1, hidden_size=hidden_size, num_layers=num_layers)
model, history = train_model(model, train_loader, val_loader, device, epochs=epochs, lr=lr)

# plot loss
plt.figure(figsize=(8,4))
plt.plot(history['train_loss'], label='train')
plt.plot(history['val_loss'], label='val')
plt.xlabel('Epoch'); plt.ylabel('MSE'); plt.legend(); plt.title('Training loss')
plt.show()

In [ ]:
# Cell 6: Forecast and evaluate on test horizon
seed_start = len(series) - test_days - seq_len
seed_seq = all_scaled[seed_start:seed_start+seq_len]
preds = iterative_forecast(model, seed_seq, steps=test_days, device=device, scaler=scaler)

true = test.values
mae = np.mean(np.abs(true - preds))
rmse = np.sqrt(np.mean((true - preds)**2))
mape = np.mean(np.abs((true - preds)/np.where(true==0, np.nan, true))) * 100
print(f'GRU MAE={mae:.4f} RMSE={rmse:.4f} MAPE={mape:.2f}%')

pred_series = pd.Series(preds, index=test.index)
plt.figure(figsize=(12,6))
plt.plot(train.index, train.values, color='gray', label='train')
plt.plot(test.index, test.values, color='black', label='test')
plt.plot(pred_series.index, pred_series.values, color='tab:blue', label='GRU')
plt.legend(); plt.title('GRU forecast vs test')
plt.show()